In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 데이터 로드
file_path = 'data/소매판매액지수raw.csv'

# 'utf-8' 오류가 났으므로 'cp949'로 시도합니다.
# 만약 'cp949'도 안 되면 'euc-kr'을 써보세요.
try:
    df_raw = pd.read_csv(file_path, encoding='cp949')
except UnicodeDecodeError:
    df_raw = pd.read_csv(file_path, encoding='euc-kr')

In [ ]:
# 데이터 전처리 (가로 -> 세로 변환)
dates = df_raw.columns[1:]
values = df_raw.iloc[1, 1:].values

In [5]:
df = pd.DataFrame({'Retail_Index': values}, index=dates)
df.index.name = '날짜'

In [6]:
# 날짜 형식 변환 및 정렬
df.index = df.index.str.replace(r'[^\d.]', '', regex=True)
df.index = pd.to_datetime(df.index, format='%Y.%m')
df = df.sort_index()

In [7]:
# 숫자 변환
df['Retail_Index'] = pd.to_numeric(df['Retail_Index'])

In [ ]:
#  12개월 누적 로그 변화율 및 Z-Score 계산
df['log_index'] = np.log(df['Retail_Index'])
df['12m_log_return'] = df['log_index'].diff(12)

In [9]:
mean_val = df['12m_log_return'].mean()
std_val = df['12m_log_return'].std()

In [10]:
df['z_score'] = (df['12m_log_return'] - mean_val) / std_val

In [11]:
# 저장
output_filename = 'data/Z_score지수/Retail_Sales_Z_Score.csv'
df.to_csv(output_filename, encoding='utf-8-sig')
print(f"'{output_filename}' 파일이 저장되었습니다.")

'data/Z_score지수/Retail_Sales_Z_Score.csv' 파일이 저장되었습니다.


In [12]:
# 결과 확인
print(df[['Retail_Index', '12m_log_return', 'z_score']].tail())

            Retail_Index  12m_log_return   z_score
날짜                                                
2025-08-01         102.3       -0.000977 -0.467792
2025-09-01         106.9        0.023664  0.296511
2025-10-01         105.1        0.000952 -0.407961
2025-11-01         106.2        0.006613 -0.232365
2025-12-01         108.9        0.010152 -0.122588
